In [1]:
# Ankit Kumar Sahoo
# 19IM10006

from numpy.random import randint
from numpy.random import rand
import pandas as pd
import random as rd

from itertools import combinations
import math
D = 60
K = 0.15
rt = 0.01
#format of the data 
#[Loan Size -> L, interest -> rl,rating, loss -> lambda]
#Customers details provided in the question 
customers = [[10, 0.021, 'AAA', 0.0002],
             [25, 0.022, 'BB', 0.0058],
             [4, 0.021, 'A', 0.001],
             [11, 0.027, 'AA', 0.0003],
             [18, 0.025, 'BBB', 0.0024],
             [3, 0.026, 'AAA', 0.0002],
             [17, 0.023, 'BB', 0.0058],
             [15, 0.021, 'AAA', 0.0002],
             [9, 0.028, 'A', 0.001],
             [10, 0.022, 'A', 0.001]
            ]

In [2]:
# objective function (Basically fitness function)
# rl * L  - lambda +  rt*Transactional cost  - 0.009*D - lambda
def fitness(x):
  sum  = -0.009*D
  for i in range(len(x)):
    if x[i] == 1:
      transaction_cost = (1-K)*D - customers[i][0]
      sum = sum + customers[i][1]*customers[i][0] - 2*customers[i][3] + 0.01 * transaction_cost
  return (sum)

In [3]:
def validation(x):
  sum = 0
  for i in range(len(x)):
    if x[i] == 1:
      sum = sum + customers[i][0] 
  return (sum)

In [4]:
# Roulette wheel selection method
# Generating the cummulative prob of selecting the chromosome based on fitness
# Selecting the individual using rando numbers 
def roulette_select(population, num):
    fitnesses = [fitness(i) for i in population]
    total_fitness = float(sum(fitnesses))
    rel_fitness = [f/total_fitness for f in fitnesses]
    probs = [sum(rel_fitness[:i+1]) for i in range(len(rel_fitness))]
    new_population = []
    for n in range(num):
        r = rand()
        for (i, individual) in enumerate(population):
            if r <= probs[i]:
                new_population.append(individual)
                break
    return new_population

In [5]:
# Single point crossover where R_CROSS is crossover probability
# Implemented genetic validation as well so that newly generating chromosomes follows constraints
def crossover(p1, p2, r_cross):
  c1, c2 = p1.copy(), p2.copy() 
  if rand() < r_cross:
    pt = randint(1, len(p1)-2)
    c1 = p1[:pt] + p2[pt:]
    c2 = p2[:pt] + p1[pt:]
  if validation(c1) > (1-K)*D :
    c1 = p1
  if validation(c2) > (1-K)*D :
    c2 = p2
  return [c1, c2]

In [6]:
# Implemented mutation using mutation prob
# Implemented mutation validation as well
def mutation(bitstr, r_mut):
  for i in range(len(bitstr)):
    if rand() < r_mut:
      bitstr[i] = 1 - bitstr[i]   
      if validation(bitstr)>((1-K)*D) :
        bitstr[i] = 1 - bitstr[i]

In [7]:
initial_sol=randint(0, 2, 10).tolist()

In [8]:
# Generating neighbourhood solution
def SwapMove(solution, i ,j):
    solution = solution.copy()
    solution[i], solution[j] = solution[j], solution[i]
    return solution

In [9]:
# Tabu structure
def get_tabuestructure():
    #dict = {{3,4}:{'ITERATIONS': 0,'FITNESS': 0}}
    dict = {}
    instance_dict = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
    for swap in combinations(instance_dict, 2):
        dict[swap] = {'ITERATIONS': 0, 'FITNESS': 0}
    return dict

In [10]:
def TabuSearch(initial_sol):
    tenure=4
    tabu_structure = get_tabuestructure() 
    best_solution=initial_sol
    best_objvalue=fitness(initial_sol)
    current_solution=initial_sol
    current_objvalue=fitness(initial_sol)
    iter = 1
    Terminate = 0
    while  Terminate < 10 and iter < 100:
        for move in tabu_structure:
            candidate_solution = SwapMove(current_solution, move[0], move[1])
            candidate_objvalue = fitness(candidate_solution)
            #validation
            if(validation(candidate_solution)<= (1-K)*D):
              tabu_structure[move]['FITNESS'] = candidate_objvalue
            else:
              tabu_structure[move]['FITNESS'] = 0
        while True:
            # select the move with the Highest ObjValue in the neighborhood 
            best_move = max(tabu_structure, key =lambda x: tabu_structure[x]['FITNESS'])
            MoveValue = tabu_structure[best_move]["FITNESS"]
            tabu_time = tabu_structure[best_move]["ITERATIONS"]
            # Not Tabu
            if tabu_time < iter:
                # make the move
                current_solution = SwapMove(current_solution, best_move[0], best_move[1])
                current_objvalue = fitness(current_solution)
                # Best Improving move
                if MoveValue > best_objvalue:
                    best_solution = current_solution
                    best_objvalue = current_objvalue
                    Terminate = 0
                else:
                    Terminate += 1
                tabu_structure[best_move]['ITERATIONS'] = iter + tenure
                iter += 1
                break
            # If tabu
            else:
                # Aspiration
                if MoveValue > best_objvalue:
                    # make the move
                    current_solution = SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = fitness(current_solution)
                    best_solution = current_solution
                    best_objvalue = current_objvalue
                    Terminate = 0
                    iter += 1
                    break
                else:
                    tabu_structure[best_move]["FITNESS"] = 0
                    continue
    return best_solution

In [11]:
# Genetic Algorithm
def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
    pop = []
    i = 1
    while i <= n_pop:
        curr = randint(0, 2, n_bits).tolist()
        if validation(curr)<= (1-K)*D:
          pop.append(curr)
          i = i +1
    best,best_eval = 0, objective(pop[0])
    for gen in range(n_iter):
        scores = [objective(c) for c in pop]
        for i in range(n_pop):
          if scores[i] > best_eval:
            best, best_eval = pop[i], scores[i]
        print("Generation %d: New Best fitness(%s) = %.3f" % (gen+1,  best, best_eval))
        selected = roulette_select(pop,n_pop-1)
        initial_sol1 = selected[0]
        selected.append(TabuSearch(initial_sol1))
        children = list()
        for i in range(0, n_pop, 2):
          p1, p2 = selected[i], selected[i+1]
          for c in crossover(p1, p2, r_cross):
            mutation(c, r_mut)
            children.append(c)
        pop = children
    return [best, best_eval]

In [12]:
# total iterations
n_iter = 100
# bits
n_bits = 10
# population size
n_pop = 10
# crossover rate
r_cross = 0.8
# mutation rate
r_mut = 0.05
best, score = genetic_algorithm(fitness, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Genetic Algorithm - Tabu Search Implemented')
print('fitness(%s) = %f' % (best, score))

Generation 1: New Best fitness([1, 0, 1, 0, 0, 1, 0, 1, 1, 0]) = 2.534
Generation 2: New Best fitness([1, 0, 0, 0, 1, 1, 0, 0, 1, 1]) = 2.710
Generation 3: New Best fitness([1, 0, 0, 0, 1, 1, 0, 0, 1, 1]) = 2.710
Generation 4: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 5: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 6: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 7: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 8: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 9: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 10: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 11: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 12: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 13: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
Generation 14: New Best fitness([0, 0, 0, 1, 1, 1, 0, 0, 1, 1]) = 2.787
G

In [13]:
validation(best)

47